In [ ]:
!pip install google-cloud-storage -q

from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import storage
import os

# Initialize GCS client
client = storage.Client(project='bug-sync-467815')  # Replace with your project ID
bucket_name = 'sentinel-kitti-data'
bucket = client.bucket(bucket_name)

# List available sequences
print("Available sequences in GCS:")
blobs = bucket.list_blobs(prefix='sequences/')
sequences = set()
for blob in blobs:
    parts = blob.name.split('/')
    if len(parts) > 1:
        sequences.add(parts[1])
print(sorted(sequences))


Available sequences in GCS:
['05', '07', '08', '09', '14', '15', '19']


In [13]:
import os
import sys
from google.colab import drive

# 1. Mount Drive and Set Paths
drive.mount('/content/drive')
PROJECT_ROOT = '/content/drive/Othercomputers/My_Mac/sentinel'
SRC_PATH = os.path.join(PROJECT_ROOT, 'src/python')
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)
os.chdir(PROJECT_ROOT)

# 2. Authenticate with GCS (Important!)
from google.colab import auth
auth.authenticate_user()

# 3. Import and Run
from utils.gcs_data_loader import create_dataloaders

BUCKET_NAME = 'sentinel-kitti-data'

# Before creating data loaders, clear the old cache to be safe
!rm -rf /content/kitti_cache/*

print("Creating data loaders with correct sequences...")
train_loader, val_loader = create_dataloaders(bucket_name=BUCKET_NAME, batch_size=4)

print("\nTesting the data loader...")
points, labels = next(iter(train_loader))

print(f"Successfully loaded one batch!")
print(f"Points tensor shape: {points.shape}")
print(f"Labels tensor shape: {labels.shape}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Creating data loaders with correct sequences...
Building file list from GCS bucket (this will happen only once)...
Dataset initialized with 5453 samples
Loading file list from cache: /content/kitti_cache/file_list_cache.pkl
Dataset initialized with 5453 samples

Testing the data loader...


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/collate.py", line 398, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/collate.py", line 212, in collate
    collate(samples, collate_fn_map=collate_fn_map)
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/collate.py", line 155, in collate
    r

In [10]:
from .content.drive.Othercomputers.My_Mac.sentinel.src.python.utils.gcs_data_loader.py import create_dataloaders

BUCKET_NAME = 'sentinel-kitti-data'  # Or whatever your bucket is named

print("Creating data loaders...")
train_loader, val_loader = create_dataloaders(bucket_name=BUCKET_NAME, batch_size=4)

print("\nTesting the data loader...")
# Get one batch of data to see if it works
points, labels = next(iter(train_loader))

print(f"Successfully loaded one batch!")
print(f"Points tensor shape: {points.shape}") # Expected: [4, Num_Points, 4]
print(f"Labels tensor shape: {labels.shape}") # Expected: [4, Num_Points]

ImportError: attempted relative import with no known parent package